# FIN Model

In [225]:
import numpy as np
from numpy import random
import pandas as pd
from scipy.signal import cwt,ricker
from scipy import stats
from sklearn.preprocessing import StandardScaler,normalize
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.optimizers
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.models import load_model
from tensorflow.keras.layers import Flatten,concatenate,Layer,Dense,Activation,MaxPooling2D,Dropout,Conv2D,BatchNormalization,Reshape,UpSampling2D,ZeroPadding2D

In [205]:
random.seed(222)

# Task 1

Train two feature imitating networks that take as input a fixed length sequence and provide as output (1) the mean and (2) the standard deviation of a fixed length input; a deep feedforward architecture should be sufficient for this.

In [206]:
# Z-scored np array
# T x Ch x Epochs
N = 100 # Fixed number of length data
E = 3000 # Size
input_arr = np.random.randint(10, size=(N,1, E) )
scaled_arr = np.zeros(input_arr.shape)

#print(input_arr)

for ii in range(input_arr.shape[1]):
#     scaler = StandardScaler()
#     scaled_arr[:,ii,:] = scaler.fit_transform(input_arr[:,ii,:])
    scaled_arr[:,ii,:] = input_arr[:,ii,:]
    
#print(scaled_arr.shape)
#print(scaled_arr)

# move the axis of the matrix as as # Epochs X T X Ch

scaled_arr = np.moveaxis(scaled_arr,2,0)
print(scaled_arr.shape)

do_cwt = False
if do_cwt:
    # Continuous wavelet transform
    flat_data = np.squeeze(scaled_arr)
    scaled_arr_cwt = []
    for ii in range(flat_data.shape[0]):
        widths = np.arange(1, 31)
        cwtmatr = cwt(flat_data[ii], ricker, widths)
        scaled_arr_cwt.append(np.moveaxis(cwtmatr,1,0))
    scaled_arr3d = np.stack(scaled_arr_cwt)
else:
    scaled_arr3d = scaled_arr
    
print(scaled_arr)



(3000, 100, 1)
[[[6.]
  [6.]
  [7.]
  ...
  [8.]
  [0.]
  [2.]]

 [[3.]
  [9.]
  [4.]
  ...
  [6.]
  [1.]
  [9.]]

 [[2.]
  [6.]
  [2.]
  ...
  [3.]
  [4.]
  [6.]]

 ...

 [[2.]
  [5.]
  [9.]
  ...
  [6.]
  [5.]
  [4.]]

 [[8.]
  [1.]
  [9.]
  ...
  [3.]
  [7.]
  [0.]]

 [[4.]
  [2.]
  [0.]
  ...
  [5.]
  [2.]
  [5.]]]


In [207]:
# Will use Mean for training, so calculate the Mean of the data
def Mean(data):
    return np.mean(data,axis=1)

#squeeze the data in
feature_res = np.squeeze(Mean(scaled_arr))

# the predict result should be
print(np.mean(scaled_arr))
print(feature_res)
print(feature_res.shape)

4.50003
[4.56 4.53 4.72 ... 4.62 4.23 4.7 ]
(3000,)


In [208]:
# Make FIN
def make_FIN(topology,size3D,N,name="my-FIN"):
    input_layer = keras.Input(shape=(N,size3D))
    x = Flatten()(input_layer)
    for layer in topology['layers']:
        x = Dense(layer['output_shape'], activation='relu')(x)
        if layer['BatchNorm']:
            x = BatchNormalization(axis=1)(x)
    fin_out = Dense(1, activation='linear')(x)
    fin = keras.Model(input_layer, fin_out, name=name)
    return fin

topology = np.load('RandomTopology.npy', allow_pickle=True)[1]


fin = make_FIN(topology,1,N)


In [209]:
#learning_rate=topology['lr']
learning_rate= 0.0005
momentum=topology['momentum']
nesterov=topology['nesterov']
b1 = topology['b1']
b2 = topology['b2']
if topology['opt'] == 'SGD':
    grad_desc_algorithm = tf.keras.optimizers.SGD(learning_rate=learning_rate,momentum=momentum,nesterov=nesterov)
if topology['opt'] == 'ADAM':
    grad_desc_algorithm = tf.keras.optimizers.Adam(learning_rate=learning_rate,beta_1=b1,beta_2=b2)
    
#es = EarlyStopping(monitor='accuracy', mode='min', verbose=1,patience=3)

fin.compile(optimizer = grad_desc_algorithm,loss = 'mean_squared_error')

print(scaled_arr3d)
print(feature_res)

loss_d = fin.fit(scaled_arr3d, feature_res, 
            epochs = 1000,
            batch_size = 500,
            verbose   = 1)    


[[[6.]
  [6.]
  [7.]
  ...
  [8.]
  [0.]
  [2.]]

 [[3.]
  [9.]
  [4.]
  ...
  [6.]
  [1.]
  [9.]]

 [[2.]
  [6.]
  [2.]
  ...
  [3.]
  [4.]
  [6.]]

 ...

 [[2.]
  [5.]
  [9.]
  ...
  [6.]
  [5.]
  [4.]]

 [[8.]
  [1.]
  [9.]
  ...
  [3.]
  [7.]
  [0.]]

 [[4.]
  [2.]
  [0.]
  ...
  [5.]
  [2.]
  [5.]]]
[4.56 4.53 4.72 ... 4.62 4.23 4.7 ]
Epoch 1/1000
6/6 [==============================] - 1s 26ms/step - loss: 19.7734
Epoch 2/1000
6/6 [==============================] - 0s 36ms/step - loss: 14.9043
Epoch 3/1000
6/6 [==============================] - 0s 21ms/step - loss: 6.6106
Epoch 4/1000
6/6 [==============================] - 0s 37ms/step - loss: 1.8952
Epoch 5/1000
6/6 [==============================] - 0s 14ms/step - loss: 1.3231
Epoch 6/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.9784
Epoch 7/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.7188
Epoch 8/1000
6/6 [==============================] - 0s 25ms/step - loss: 0.6078
Epoch 9/1000
6/6

6/6 [==============================] - 0s 19ms/step - loss: 0.0032
Epoch 99/1000
6/6 [==============================] - 0s 35ms/step - loss: 0.0021
Epoch 100/1000
6/6 [==============================] - 0s 29ms/step - loss: 0.0052
Epoch 101/1000
6/6 [==============================] - 0s 18ms/step - loss: 0.0031
Epoch 102/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0017
Epoch 103/1000
6/6 [==============================] - 0s 25ms/step - loss: 0.0084
Epoch 104/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0089
Epoch 105/1000
6/6 [==============================] - 0s 45ms/step - loss: 0.0053
Epoch 106/1000
6/6 [==============================] - 0s 22ms/step - loss: 0.0038
Epoch 107/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0032
Epoch 108/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0044
Epoch 109/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0045
Epoch 110/1000
6/6 [===============

6/6 [==============================] - 0s 8ms/step - loss: 0.0013
Epoch 199/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0026
Epoch 200/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0043
Epoch 201/1000
6/6 [==============================] - 0s 26ms/step - loss: 0.0065
Epoch 202/1000
6/6 [==============================] - 0s 35ms/step - loss: 0.0022
Epoch 203/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0049
Epoch 204/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0018
Epoch 205/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0023
Epoch 206/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0039
Epoch 207/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0024
Epoch 208/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0068
Epoch 209/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0022
Epoch 210/1000
6/6 [=================

6/6 [==============================] - 0s 9ms/step - loss: 0.0012
Epoch 299/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0015
Epoch 300/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0015
Epoch 301/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.0025
Epoch 302/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0013
Epoch 303/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0017
Epoch 304/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0031
Epoch 305/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0027
Epoch 306/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0035
Epoch 307/1000
6/6 [==============================] - 0s 18ms/step - loss: 0.0023
Epoch 308/1000
6/6 [==============================] - 0s 25ms/step - loss: 0.0024
Epoch 309/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0033
Epoch 310/1000
6/6 [================

6/6 [==============================] - 0s 10ms/step - loss: 0.0019
Epoch 399/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0034
Epoch 400/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0055
Epoch 401/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.0022
Epoch 402/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0014
Epoch 403/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0011
Epoch 404/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0023
Epoch 405/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0022
Epoch 406/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0030
Epoch 407/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0030
Epoch 408/1000
6/6 [==============================] - 0s 22ms/step - loss: 0.0059
Epoch 409/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0015
Epoch 410/1000
6/6 [====================

...

6/6 [==============================] - 0s 15ms/step - loss: 0.0064
Epoch 599/1000
6/6 [==============================] - 0s 19ms/step - loss: 0.0033
Epoch 600/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0027
Epoch 601/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0052
Epoch 602/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0021
Epoch 603/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0010
Epoch 604/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0035
Epoch 605/1000
6/6 [==============================] - 0s 20ms/step - loss: 0.0057
Epoch 606/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0073
Epoch 607/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0016
Epoch 608/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0036
Epoch 609/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0014
Epoch 610/1000
6/6 [===================

6/6 [==============================] - 0s 21ms/step - loss: 0.0040
Epoch 699/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0015
Epoch 700/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0024
Epoch 701/1000
6/6 [==============================] - 0s 15ms/step - loss: 9.8832e-04
Epoch 702/1000
6/6 [==============================] - 0s 13ms/step - loss: 7.9633e-04
Epoch 703/1000
6/6 [==============================] - 0s 7ms/step - loss: 0.0012
Epoch 704/1000
6/6 [==============================] - 0s 7ms/step - loss: 0.0035
Epoch 705/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0030
Epoch 706/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0017
Epoch 707/1000
6/6 [==============================] - 0s 7ms/step - loss: 0.0013
Epoch 708/1000
6/6 [==============================] - 0s 12ms/step - loss: 8.6774e-04
Epoch 709/1000
6/6 [==============================] - 0s 11ms/step - loss: 0.0014
Epoch 710/1000
6/6 [====

6/6 [==============================] - 0s 8ms/step - loss: 0.0031
Epoch 798/1000
6/6 [==============================] - 0s 19ms/step - loss: 0.0024
Epoch 799/1000
6/6 [==============================] - 0s 8ms/step - loss: 8.3703e-04
Epoch 800/1000
6/6 [==============================] - 0s 8ms/step - loss: 9.4914e-04
Epoch 801/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0023
Epoch 802/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0014
Epoch 803/1000
6/6 [==============================] - 0s 19ms/step - loss: 0.0018
Epoch 804/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0034
Epoch 805/1000
6/6 [==============================] - 0s 10ms/step - loss: 0.0019
Epoch 806/1000
6/6 [==============================] - 0s 15ms/step - loss: 0.0026
Epoch 807/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0045
Epoch 808/1000
6/6 [==============================] - 0s 12ms/step - loss: 0.0028
Epoch 809/1000
6/6 [==========

6/6 [==============================] - 0s 9ms/step - loss: 0.0020
Epoch 897/1000
6/6 [==============================] - 0s 9ms/step - loss: 0.0022
Epoch 898/1000
6/6 [==============================] - 0s 16ms/step - loss: 0.0029
Epoch 899/1000
6/6 [==============================] - 0s 22ms/step - loss: 0.0011
Epoch 900/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0014
Epoch 901/1000
6/6 [==============================] - 0s 29ms/step - loss: 6.6276e-04
Epoch 902/1000
6/6 [==============================] - 0s 14ms/step - loss: 0.0027
Epoch 903/1000
6/6 [==============================] - 0s 17ms/step - loss: 0.0012
Epoch 904/1000
6/6 [==============================] - 0s 8ms/step - loss: 8.7575e-04
Epoch 905/1000
6/6 [==============================] - 0s 18ms/step - loss: 0.0013
Epoch 906/1000
6/6 [==============================] - 0s 24ms/step - loss: 0.0023
Epoch 907/1000
6/6 [==============================] - 0s 13ms/step - loss: 9.4493e-04
Epoch 908/1000
6/6 [===

Epoch 995/1000
6/6 [==============================] - 0s 23ms/step - loss: 0.0012
Epoch 996/1000
6/6 [==============================] - 0s 8ms/step - loss: 0.0019
Epoch 997/1000
6/6 [==============================] - 0s 8ms/step - loss: 8.0623e-04
Epoch 998/1000
6/6 [==============================] - 0s 8ms/step - loss: 9.3140e-04
Epoch 999/1000
6/6 [==============================] - 0s 8ms/step - loss: 9.2531e-04
Epoch 1000/1000
6/6 [==============================] - 0s 35ms/step - loss: 7.8232e-04


In [264]:
# Predirect_arr = np.random.randint(10, size=(1,100, 1)).T

# fin.predict(Predirect_arr)
# print(np.mean(Predirect_arr))
# print(fin.predict(Predirect_arr))


4.41
[[4.4110794]]


In [233]:
fin.save("./Models/Modelstask1Mean.h5")

In [234]:
task1MeanModel = load_model('./Models/Modelstask1Mean.h5')


In [252]:
Predirect_arr = np.random.randint(10, size=(1,100, 1)).T
print(np.mean(Predirect_arr))
task1MeanModel.predict(Predirect_arr)


4.45


array([[4.5799036]], dtype=float32)